In [1]:
import json
import re
import argparse
import os
from itertools import chain
import pandas as pd
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification, LongformerTokenizerFast
from datasets import Dataset
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
INFERENCE_MAX_LENGTH = [1024, 1536]
INFERENCE_STRIDE = 256
TRAINING_MODEL_PATH = [
    '/kaggle/input/pii-data-detection-models/checkpoint-900',
    '/kaggle/input/full-model/pious-sweep-3/checkpoint-2900'
]

In [4]:
id2label = {
    0: 'B-EMAIL',
    1: 'B-ID_NUM',
    2: 'B-NAME_STUDENT',
    3: 'B-PHONE_NUM',
    4: 'B-STREET_ADDRESS',
    5: 'B-URL_PERSONAL',
    6: 'B-USERNAME',
    7: 'I-ID_NUM',
    8: 'I-NAME_STUDENT',
    9: 'I-PHONE_NUM',
    10: 'I-STREET_ADDRESS',
    11: 'I-URL_PERSONAL',
    12: 'O'
}
label2id = {
    'B-EMAIL': 0,
    'B-ID_NUM': 1,
    'B-NAME_STUDENT': 2,
    'B-PHONE_NUM': 3,
    'B-STREET_ADDRESS': 4,
    'B-URL_PERSONAL': 5,
    'B-USERNAME': 6,
    'I-ID_NUM': 7,
    'I-NAME_STUDENT': 8,
    'I-PHONE_NUM': 9,
    'I-STREET_ADDRESS': 10,
    'I-URL_PERSONAL': 11,
    'O': 12
}
all_labels = [
    'B-EMAIL',
    'B-ID_NUM',
    'B-NAME_STUDENT',
    'B-PHONE_NUM',
    'B-STREET_ADDRESS',
    'B-URL_PERSONAL',
    'B-USERNAME',
    'I-ID_NUM',
    'I-NAME_STUDENT',
    'I-PHONE_NUM',
    'I-STREET_ADDRESS',
    'I-URL_PERSONAL',
    'O'
]

In [5]:
df = pd.read_json("/kaggle/input/pii-detection-removal-from-educational-data/test.json")


def get_labels(word_ids, word_labels):
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        else:
            label_ids.append(label2id[word_labels[word_idx]])
    return label_ids

# Tokenize texts, possibly generating more than one tokenized sample for each text


def tokenize(df, inference_max_length, to_tensor=True, with_labels=True):

    # This is what's different from a longformer
    # Read the parameters with attention
    encoded = tokenizer(df['tokens'].tolist(),
                        is_split_into_words=True,
                        return_overflowing_tokens=True,
                        stride=INFERENCE_STRIDE,
                        max_length=inference_max_length,
                        padding="max_length",
                        truncation=True)

    if with_labels:
        encoded['labels'] = []

    encoded['wids'] = []
    n = len(encoded['overflow_to_sample_mapping'])
    for i in range(n):

        # Map back to original row
        text_idx = encoded['overflow_to_sample_mapping'][i]

        # Get word indexes (this is a global index that takes into consideration the chunking :D )
        word_ids = encoded.word_ids(i)

        if with_labels:
            # Get word labels of the full un-chunked text
            word_labels = df['labels'].iloc[text_idx]

            # Get the labels associated with the word indexes
            label_ids = get_labels(word_ids, word_labels)
            encoded['labels'].append(label_ids)
        encoded['wids'].append([w if w is not None else -1 for w in word_ids])

    if to_tensor:
        encoded = {key: torch.as_tensor(val) for key, val in encoded.items()}
    return encoded


class PIIDataset(Dataset):
    def __init__(self, tokenized_ds):
        self.data = tokenized_ds

    def __getitem__(self, index):
        item = {k: self.data[k][index] for k in self.data.keys()}
        return item

    def __len__(self):
        return len(self.data['input_ids'])

In [6]:
def inferenceV3(df, dl, thres=0.9):

    # These 2 dictionaries will hold text-level data
    # Helping in the merging process by accumulating data
    # Through all the chunks

    token_pred = defaultdict(lambda: defaultdict(int))
    seen_words_idx = defaultdict(set)

    for batch in tqdm(dl):
        ids = batch["input_ids"].to("cuda")
        mask = batch["attention_mask"].to("cuda")
        preds = model(ids, attention_mask=mask, return_dict=False)[0].cpu().detach().numpy()
        preds_softmax = np.exp(preds) / np.sum(np.exp(preds), axis=2).reshape(preds.shape[0], preds.shape[1], 1)

        del ids, mask

        # Go over each prediction, getting the text_id reference

        for k, (chunk_preds, text_id) in enumerate(zip(preds_softmax, batch['overflow_to_sample_mapping'].tolist())):
            # The word_ids are absolute references in the original text
            word_ids = batch['wids'][k].numpy()

            for idx, word_idx in enumerate(word_ids):
                if word_idx != -1 and word_idx not in seen_words_idx[text_id]:
                    token_pred[text_id][word_idx] += chunk_preds[idx]
                    seen_words_idx[text_id].add(word_idx)
    return token_pred

In [7]:
def inferenceV4(df, dl):

    # These 2 dictionaries will hold text-level data
    # Helping in the merging process by accumulating data
    # Through all the chunks

    token_pred = defaultdict(lambda: defaultdict(int))
    token_cnt = defaultdict(lambda: defaultdict(int))

    for batch in tqdm(dl):
        ids = batch["input_ids"].to("cuda")
        mask = batch["attention_mask"].to("cuda")
        preds = model(ids, attention_mask=mask, return_dict=False)[0].cpu().detach().numpy()
        preds_softmax = np.exp(preds) / np.sum(np.exp(preds), axis=2).reshape(preds.shape[0], preds.shape[1], 1)

        del ids, mask

        # Go over each prediction, getting the text_id reference

        for k, (chunk_preds, text_id) in enumerate(zip(preds_softmax, batch['overflow_to_sample_mapping'].tolist())):
            # The word_ids are absolute references in the original text
            word_ids = batch['wids'][k].numpy()

            for idx, word_idx in enumerate(word_ids):
                if word_idx != -1:
                    token_pred[text_id][word_idx] += chunk_preds[idx]
                    token_cnt[text_id][word_idx] += 1

    for text_id in token_pred:
        for word_idx in token_pred[text_id]:
            token_pred[text_id][word_idx] /= token_cnt[text_id][word_idx]

    return token_pred

In [8]:
def inferenceV5(df, dl):

    # These 2 dictionaries will hold text-level data
    # Helping in the merging process by accumulating data
    # Through all the chunks

    token_pred = defaultdict(lambda: defaultdict(int))
    token_cnt = defaultdict(lambda: defaultdict(int))

    for batch in tqdm(dl):
        ids = batch["input_ids"].to("cuda")
        mask = batch["attention_mask"].to("cuda")
        preds = model(ids, attention_mask=mask, return_dict=False)[0].cpu().detach().numpy()
        preds_softmax = np.exp(preds) / np.sum(np.exp(preds), axis=2).reshape(preds.shape[0], preds.shape[1], 1)

        del ids, mask

        # Go over each prediction, getting the text_id reference
        seen_words_idx = set()
        for k, (chunk_preds, text_id) in enumerate(zip(preds_softmax, batch['overflow_to_sample_mapping'].tolist())):
            # The word_ids are absolute references in the original text
            word_ids = batch['wids'][k].numpy()

            for idx, word_idx in enumerate(word_ids):
                if word_idx != -1 and word_idx not in seen_words_idx:
                    seen_words_idx.add(word_idx)
                    token_pred[text_id][word_idx] += chunk_preds[idx]
                    token_cnt[text_id][word_idx] += 1

    for text_id in token_pred:
        for word_idx in token_pred[text_id]:
            token_pred[text_id][word_idx] /= token_cnt[text_id][word_idx]

    return token_pred

In [9]:
final_token_pred = defaultdict(lambda: defaultdict(int))
for idx, model_path in enumerate(TRAINING_MODEL_PATH):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenized_test = tokenize(df, inference_max_length=INFERENCE_MAX_LENGTH[idx], with_labels=False)

    test_dataset = PIIDataset(tokenized_test)
    test_dataloader = DataLoader(test_dataset, batch_size=1)
    model = AutoModelForTokenClassification.from_pretrained(
        model_path,
        num_labels=len(all_labels),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )
    model.cuda()
    token_pred = inferenceV4(df, test_dataloader)
    for text_id in token_pred:
        for word_idx in token_pred[text_id]:
            final_token_pred[text_id][word_idx] += token_pred[text_id][word_idx] / len(TRAINING_MODEL_PATH)

100%|██████████| 11/11 [00:08<00:00,  1.34it/s]


In [10]:
document, token, label = [], [], []
for text_id in final_token_pred:
    for word_idx in final_token_pred[text_id]:
        pred = final_token_pred[text_id][word_idx].argmax(-1)
        pred_without_O = final_token_pred[text_id][word_idx][:12].argmax(-1)
        if final_token_pred[text_id][word_idx][12] < 0.55:
            final_pred = pred_without_O
        else:
            final_pred = pred
        if id2label[final_pred] != 'O':
            document.append(df.loc[text_id, "document"])
            token.append(word_idx)
            label.append(id2label[final_pred])

pred_df = pd.DataFrame({
    "document": document,
    "token": token,
    "label": label
})

In [11]:
def score(row):
    doc = row.document
    tok = row.token
    doc_idx = df.query("document == @doc").index[0]
    return token_pred[doc_idx][tok][label2id[row.label]]

In [12]:
pred_df["score"] = pred_df.apply(lambda x: score(x), axis=1)

Postprocess

In [13]:
df = df[['document', 'tokens']].copy()

In [14]:
df = df.explode(['tokens']).reset_index(drop=True).rename(columns={'tokens': 'token'})

In [15]:
df['token_str'] = df['token']
df['token'] = df.groupby('document').cumcount()

In [16]:
new_pred_df = pd.merge(df, pred_df[['document', 'token', 'label', "score"]], on=['document', 'token'], how='left')
new_pred_df['label'] = new_pred_df['label'].fillna('O')

In [17]:
def pp(new_pred_df):
    df = new_pred_df.copy()
    i = 0
    while i < len(df):
        st = i
        doc = df.loc[st, "document"]
        tok = df.loc[st, "token"]
        pred_tok = df.loc[st, "label"]
        if pred_tok == 'O':
            i += 1
            continue
        lab = pred_tok.split('-')[1]
        cur_doc = doc
        cur_lab = lab
        last_tok = tok
        cur_tok = last_tok
        # prefix = []
        while i < len(df) and cur_doc == doc and cur_lab == lab and last_tok == cur_tok:
            # prefix.append(pred_tok.split('-')[0])
            last_tok = cur_tok + 1
            i += 1
            cur_doc = df.loc[i, "document"]
            cur_tok = df.loc[i, "token"]
            if i >= len(df) or df.loc[i, "label"] == 'O':
                break
            cur_lab = df.loc[i, "label"].split('-')[1]
        # exception
        if st - 2 >= 0 and df.loc[st - 2, "document"] == df.loc[st, "document"] and df.loc[st - 1, "token_str"] == '\n' and df.loc[st - 2, "label"] != 'O' and df.loc[st - 2, "label"].split('-')[1] == lab:
            df.loc[st - 1, "label"] = 'I-' + lab
            df.loc[st - 1, "score"] = 1
            for j in range(st, i):
                if df.loc[j, "label"] != 'I-' + lab:
                    df.loc[j, "score"] = 1
                    df.loc[j, "label"] = 'I-' + lab
            continue

        # fix
        for j in range(st, i):
            if j == st:
                if df.loc[j, "label"] != 'B-' + lab:
                    df.loc[j, "score"] = 1
                    df.loc[j, "label"] = 'B-' + lab
            else:
                if df.loc[j, "label"] != 'I-' + lab:
                    df.loc[j, "score"] = 1
                    df.loc[j, "label"] = 'I-' + lab
#         print(df.loc[st:i,:])
        if lab == 'NAME_STUDENT' and any(len(item) == 2 and item[0].isupper() and item[1] == "." for item in df.loc[st:i-1, 'token_str']):
            for j in range(st, i):
                df.loc[j, "score"] = 0
                df.loc[j, "label"] = 'O'

    return df

In [18]:
new_pred_df = pp(new_pred_df)

In [19]:
new_pred_df

,document,token,token_str,label,score
0,7,0,Design,O,NaN
1,7,1,Thinking,O,NaN
2,7,2,for,O,NaN
3,7,3,innovation,O,NaN
4,7,4,reflexion,O,NaN
...,...,...,...,...,...
8500,123,1689,(,O,NaN
8501,123,1690,https://www.melessa.uni-,O,NaN
8502,123,1691,muenchen.de/team/vorstandssprecher/schmidt/pub...,O,NaN
8503,123,1692,),O,NaN


In [20]:
new_pred_df = new_pred_df.query("label != 'O'").reset_index(drop=True)

In [21]:
thresholds = {
    'B-EMAIL': 0.99,
    'B-ID_NUM': 0.4,
    'B-NAME_STUDENT': 0,
    'B-PHONE_NUM': 0.99,
    'B-STREET_ADDRESS': 0.99,
    'B-URL_PERSONAL': 0.99,
    'B-USERNAME': 0.8,
    'I-ID_NUM': 0.8,
    'I-NAME_STUDENT': 0.4,
    'I-PHONE_NUM': 0.99,
    'I-STREET_ADDRESS': 0.99,
    'I-URL_PERSONAL': 0
}

In [24]:
rows_to_delete = []
for idx, row in new_pred_df.iterrows():
    if row.label == 'I-PHONE_NUM':
        if row.token_str == ')':
            rows_to_delete.append(idx)
        elif not bool(re.search(r'\d', row.token_str)):
            rows_to_delete.append(idx)
    elif row.label == 'B-EMAIL':
        if '@' not in row.token_str:
            rows_to_delete.append(idx)

In [25]:
new_pred_df = new_pred_df.drop(rows_to_delete, axis=0, inplace=False)

In [26]:
new_pred_df["row_id"] = list(range(len(new_pred_df)))

In [27]:
new_pred_df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)

In [28]:
new_pred_df[["row_id", "document", "token", "label", "token_str"]]

,row_id,document,token,label,token_str
0,0,7,9,B-NAME_STUDENT,Nathalie
1,1,7,10,I-NAME_STUDENT,Sylla
2,2,7,482,B-NAME_STUDENT,Nathalie
3,3,7,483,I-NAME_STUDENT,Sylla
4,4,7,741,B-NAME_STUDENT,Nathalie
5,5,7,742,I-NAME_STUDENT,Sylla
6,6,10,0,B-NAME_STUDENT,Diego
7,7,10,1,I-NAME_STUDENT,Estrada
8,8,10,464,B-NAME_STUDENT,Diego
9,9,10,465,I-NAME_STUDENT,Estrada
